# Softmax Classifier Animal Classification
- [Softmax를 사용하여 Zoo 데이터 동물 분류](https://github.com/deeplearningzerotoall/TensorFlow/blob/master/tf_2.x/lab-06-2-softmax_zoo_classifier-eager.ipynb)

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

tf.random.set_seed(777) # for reproducibility

2.4.1


In [13]:
xy = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/NAVER boostcourse/Deep Learning with Tensorflow/Part1_Basic ML/data/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1] # 16 features
y_data = xy[:, -1] # 0~6 classes

nb_classes = 7

# Make Y data as one-hot shape
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)
print(Y_one_hot)

(101, 16) (101, 7)
tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0.

In [30]:
# Weight and bias setting
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes, )), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit(X))

def cost_function(X, Y):
    logits = logit(X)
    # Cross-Entropy cost function
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, from_logits=True) # # Tensorflow 2.x
    # cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot) # Tensorflow 1.x
    cost_mean = tf.reduce_mean(cost_i)
    return cost_mean

def gradient_function(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_function(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [31]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

    for epoch in range(epochs):
        grads = gradient_function(X, Y)
        optimizer.apply_gradients(zip(grads, variables))

        if (epoch == 0) or ((epoch+1) % verbose == 0):
            # print('Loss at epoch %d: %f' %(epoch+1, cost_function(X, Y).numpy()))
            acc = prediction(X, Y).numpy()
            loss = cost_function(X, Y).numpy()
            print('epoch: {} | Loss: {} | Acc: {}'.format(epoch+1, loss, acc))

fit(x_data, Y_one_hot)

epoch: 1 | Loss: 5.199698448181152 | Acc: 0.30693069100379944
epoch: 100 | Loss: 0.6987780332565308 | Acc: 0.8316831588745117
epoch: 200 | Loss: 0.42176562547683716 | Acc: 0.9108911156654358
epoch: 300 | Loss: 0.31414729356765747 | Acc: 0.9306930899620056
epoch: 400 | Loss: 0.24939119815826416 | Acc: 0.9405940771102905
epoch: 500 | Loss: 0.20510666072368622 | Acc: 0.9504950642585754
epoch: 600 | Loss: 0.17317898571491241 | Acc: 0.9603960514068604
epoch: 700 | Loss: 0.14934492111206055 | Acc: 0.9702970385551453
epoch: 800 | Loss: 0.13104161620140076 | Acc: 0.9801980257034302
epoch: 900 | Loss: 0.11664841324090958 | Acc: 0.9801980257034302
epoch: 1000 | Loss: 0.10509525239467621 | Acc: 0.9900990128517151
